# Factory Overrides

In [1]:
from pyuvm import *

class Producer(uvm_component):
    def build_phase(self):
        self.ap = uvm_analysis_port("ap", self)
        
    def run_phase(self):
        self.raise_objection()
        for nn in range(5):
            self.ap.write(nn)
        self.drop_objection()
        
class Consumer(uvm_subscriber):
    def write(self, datum):
        print(datum, end=" ")

class VertConsumer(Consumer):
    def write(self, datum):
        print(datum)

class IncConsumer(VertConsumer):
    def write(self, datum):
        print(datum+1)
        
class Env(uvm_env):
    def build_phase(self):
        self.pro  = Producer.create("pro", self)
        self.con   = Consumer.create("con", self)
        self.con2  = Consumer.create("con2", self)

    def connect_phase(self):
        self.pro.ap.connect(self.con)
        self.pro.ap.connect(self.con2)

class BaseTest(uvm_test):
    def build_phase(self):
        self.env = Env.create("env", self)

class VertTest(BaseTest):
    def build_phase(self):
        uvm_factory().set_type_override_by_type(Consumer, VertConsumer)
        super().build_phase()

class IncTest(BaseTest):
    def build_phase(self):
        uvm_factory().set_inst_override_by_type(Consumer, IncConsumer, self.get_full_name()+".env.con2")
        super().build_phase()
        
uvm_root().run_test("IncTest")
            

0 1
1 2
2 3
3 4
4 5
